In [1]:
import pandas as pd
import numpy as np
import ast
import json
from tqdm import tqdm

In [2]:
df_tie = pd.read_parquet("./ultrafeedback_ties.parquet")
print(df_tie.shape)

(16142, 8)


In [3]:
df_tie.head()

,source,prompt,chosen,chosen-rating,chosen-model,rejected,rejected-rating,rejected-model
0,evol_instruct,"Incorporating birth/death dates, accomplishmen...","[{'content': 'Incorporating birth/death dates,...",4.5,gpt-3.5-turbo,"[{'content': 'Incorporating birth/death dates,...",4.5,vicuna-33b
1,evol_instruct,Let's play a puzzle game! Can you connect the ...,[{'content': 'Let's play a puzzle game! Can yo...,5.0,llama-2-7b-chat,[{'content': 'Let's play a puzzle game! Can yo...,5.0,wizardlm-70b
2,evol_instruct,Can you write a Ruby program to classify a giv...,[{'content': 'Can you write a Ruby program to ...,4.5,ultralm-13b,[{'content': 'Can you write a Ruby program to ...,4.5,wizardlm-13b
3,evol_instruct,"Using Python, create a program that analyzes a...","[{'content': 'Using Python, create a program t...",4.5,llama-2-13b-chat,"[{'content': 'Using Python, create a program t...",4.5,llama-2-70b-chat
4,evol_instruct,Imagine a situation where you are solving a cr...,[{'content': 'Imagine a situation where you ar...,4.5,gpt-4,[{'content': 'Imagine a situation where you ar...,4.5,llama-2-13b-chat


In [4]:
df_tie["difference"] = df_tie["chosen-rating"] - df_tie["rejected-rating"]
df_tie.drop(columns=["source", "chosen-rating", "rejected-rating"], inplace=True)
df_tie.rename(columns={"chosen-model": "model_a", "rejected-model": "model_b"}, inplace=True)
df_tie["winner_model_a"] = 0
df_tie["winner_model_b"] = 0
df_tie["winner_tie"] = 1
df_tie.head()

,prompt,chosen,model_a,rejected,model_b,difference,winner_model_a,winner_model_b,winner_tie
0,"Incorporating birth/death dates, accomplishmen...","[{'content': 'Incorporating birth/death dates,...",gpt-3.5-turbo,"[{'content': 'Incorporating birth/death dates,...",vicuna-33b,0.0,0,0,1
1,Let's play a puzzle game! Can you connect the ...,[{'content': 'Let's play a puzzle game! Can yo...,llama-2-7b-chat,[{'content': 'Let's play a puzzle game! Can yo...,wizardlm-70b,0.0,0,0,1
2,Can you write a Ruby program to classify a giv...,[{'content': 'Can you write a Ruby program to ...,ultralm-13b,[{'content': 'Can you write a Ruby program to ...,wizardlm-13b,0.0,0,0,1
3,"Using Python, create a program that analyzes a...","[{'content': 'Using Python, create a program t...",llama-2-13b-chat,"[{'content': 'Using Python, create a program t...",llama-2-70b-chat,0.0,0,0,1
4,Imagine a situation where you are solving a cr...,[{'content': 'Imagine a situation where you ar...,gpt-4,[{'content': 'Imagine a situation where you ar...,llama-2-13b-chat,0.0,0,0,1


In [5]:
df_tie["chosen"][0]

array([{'content': "Incorporating birth/death dates, accomplishments, and societal impact, how can you create a program that utilizes data from a SQL database to generate a comprehensive ranking of historical figures based on their level of influence? Can you extend the scope of this inquiry by incorporating multi-dimensional analysis of individuals' contributions? Using Python code and SQL queries, can you develop a sophisticated algorithm that incorporates various weighted factors to accurately calculate each person's influence score? Once the scores have been calculated, how can you retrieve an extensive list of the top 10 most influential individuals throughout history?", 'role': 'user'},
       {'content': 'You can create a program to generate a comprehensive ranking of historical figures based on their level of influence by following these steps:\n\n1. Design and create an SQL database to store information about historical figures, including their birth/death dates, accomplishmen

In [6]:
prompts = []
response_as = []
response_bs = []

for idx, row in tqdm(df_tie.iterrows(), total=len(df_tie)):
    conversation_a = row["chosen"]
    conversation_b = row["rejected"]

    prompt_a = []
    prompt_b = []
    response_a = []
    response_b = []

    if len(conversation_a) != len(conversation_b):
        print(f"Error: conversation_a and conversation_b have different lengths at index {idx}")
    else:
        for i in range(len(conversation_a)):
            single_round_a = conversation_a[i]
            single_round_b = conversation_b[i]

            if single_round_a["role"] == "user":
                prompt_a.append(single_round_a["content"])
            elif single_round_a["role"] == "assistant":
                response_a.append(single_round_a["content"])
            else:
                print(f"Error: unexpected role at index {idx}")

            if single_round_b["role"] == "user":
                prompt_b.append(single_round_b["content"])
            elif single_round_b["role"] == "assistant":
                response_b.append(single_round_b["content"])
            else:
                print(f"Error: unexpected role at index {idx}")

        if prompt_a != prompt_b:
            print(f"Error: prompt_a and prompt_b have different lengths at index {idx}")
        else:
            prompts.append(prompt_a)
            response_as.append(response_a)
            response_bs.append(response_b)

100%|██████████| 16142/16142 [00:01<00:00, 14461.37it/s]


In [7]:
df_tie["prompt"] = prompts
df_tie["response_a"] = response_as
df_tie["response_b"] = response_bs
df_tie.head()

,prompt,chosen,model_a,rejected,model_b,difference,winner_model_a,winner_model_b,winner_tie,response_a,response_b
0,"[Incorporating birth/death dates, accomplishme...","[{'content': 'Incorporating birth/death dates,...",gpt-3.5-turbo,"[{'content': 'Incorporating birth/death dates,...",vicuna-33b,0.0,0,0,1,[You can create a program to generate a compre...,[To create a program that generates a comprehe...
1,[Let's play a puzzle game! Can you connect the...,[{'content': 'Let's play a puzzle game! Can yo...,llama-2-7b-chat,[{'content': 'Let's play a puzzle game! Can yo...,wizardlm-70b,0.0,0,0,1,"[Of course, I'd be happy to help you solve the...","[Absolutely, I'd be happy to explain how these..."
2,[Can you write a Ruby program to classify a gi...,[{'content': 'Can you write a Ruby program to ...,ultralm-13b,[{'content': 'Can you write a Ruby program to ...,wizardlm-13b,0.0,0,0,1,"[Hello! Yes, I can help you write a Ruby progr...",[This Ruby code uses the TensorFlow framework ...
3,"[Using Python, create a program that analyzes ...","[{'content': 'Using Python, create a program t...",llama-2-13b-chat,"[{'content': 'Using Python, create a program t...",llama-2-70b-chat,0.0,0,0,1,[Certainly! I'd be happy to help you with your...,[Welcome to your AI assistant! I'm here to hel...
4,[Imagine a situation where you are solving a c...,[{'content': 'Imagine a situation where you ar...,gpt-4,[{'content': 'Imagine a situation where you ar...,llama-2-13b-chat,0.0,0,0,1,[A suitable word to fill in the blank for this...,"[Sure, I'd be happy to help you solve the cros..."


In [8]:
df_tie.drop(columns=["chosen", "rejected"], inplace=True)
df_tie.head()

,prompt,model_a,model_b,difference,winner_model_a,winner_model_b,winner_tie,response_a,response_b
0,"[Incorporating birth/death dates, accomplishme...",gpt-3.5-turbo,vicuna-33b,0.0,0,0,1,[You can create a program to generate a compre...,[To create a program that generates a comprehe...
1,[Let's play a puzzle game! Can you connect the...,llama-2-7b-chat,wizardlm-70b,0.0,0,0,1,"[Of course, I'd be happy to help you solve the...","[Absolutely, I'd be happy to explain how these..."
2,[Can you write a Ruby program to classify a gi...,ultralm-13b,wizardlm-13b,0.0,0,0,1,"[Hello! Yes, I can help you write a Ruby progr...",[This Ruby code uses the TensorFlow framework ...
3,"[Using Python, create a program that analyzes ...",llama-2-13b-chat,llama-2-70b-chat,0.0,0,0,1,[Certainly! I'd be happy to help you with your...,[Welcome to your AI assistant! I'm here to hel...
4,[Imagine a situation where you are solving a c...,gpt-4,llama-2-13b-chat,0.0,0,0,1,[A suitable word to fill in the blank for this...,"[Sure, I'd be happy to help you solve the cros..."


In [9]:
def swap_columns(row):
    if np.random.rand() < 0.5:
        model_a_value = row["model_a"]
        row["model_a"] = row["model_b"]
        row["model_b"] = model_a_value

        response_a_value = row["response_a"]
        row["response_a"] = row["response_b"]
        row["response_b"] = response_a_value
    
    return row

In [10]:
df_tie = df_tie.apply(swap_columns, axis=1)
df_tie.head()

,prompt,model_a,model_b,difference,winner_model_a,winner_model_b,winner_tie,response_a,response_b
0,"[Incorporating birth/death dates, accomplishme...",gpt-3.5-turbo,vicuna-33b,0.0,0,0,1,[You can create a program to generate a compre...,[To create a program that generates a comprehe...
1,[Let's play a puzzle game! Can you connect the...,wizardlm-70b,llama-2-7b-chat,0.0,0,0,1,"[Absolutely, I'd be happy to explain how these...","[Of course, I'd be happy to help you solve the..."
2,[Can you write a Ruby program to classify a gi...,wizardlm-13b,ultralm-13b,0.0,0,0,1,[This Ruby code uses the TensorFlow framework ...,"[Hello! Yes, I can help you write a Ruby progr..."
3,"[Using Python, create a program that analyzes ...",llama-2-70b-chat,llama-2-13b-chat,0.0,0,0,1,[Welcome to your AI assistant! I'm here to hel...,[Certainly! I'd be happy to help you with your...
4,[Imagine a situation where you are solving a c...,llama-2-13b-chat,gpt-4,0.0,0,0,1,"[Sure, I'd be happy to help you solve the cros...",[A suitable word to fill in the blank for this...


In [11]:
df_tie.to_csv("kaggle-ultrafeedback-ties.csv", index=False)

In [12]:
df_tie = pd.read_csv("kaggle-ultrafeedback-ties.csv")
df_tie.head()

,prompt,model_a,model_b,difference,winner_model_a,winner_model_b,winner_tie,response_a,response_b
0,"[""Incorporating birth/death dates, accomplishm...",gpt-3.5-turbo,vicuna-33b,0.0,0,0,1,['You can create a program to generate a compr...,['To create a program that generates a compreh...
1,"[""Let's play a puzzle game! Can you connect th...",wizardlm-70b,llama-2-7b-chat,0.0,0,0,1,"[""Absolutely, I'd be happy to explain how thes...","[""Of course, I'd be happy to help you solve th..."
2,['Can you write a Ruby program to classify a g...,wizardlm-13b,ultralm-13b,0.0,0,0,1,"[""This Ruby code uses the TensorFlow framework...","['Hello! Yes, I can help you write a Ruby prog..."
3,"['Using Python, create a program that analyzes...",llama-2-70b-chat,llama-2-13b-chat,0.0,0,0,1,['Welcome to your AI assistant! I\'m here to h...,['Certainly! I\'d be happy to help you with yo...
4,['Imagine a situation where you are solving a ...,llama-2-13b-chat,gpt-4,0.0,0,0,1,"['Sure, I\'d be happy to help you solve the cr...",['A suitable word to fill in the blank for thi...


In [13]:
def ensure_json_loads(input_str):
    python_list = ast.literal_eval(input_str)
    return json.dumps(python_list, ensure_ascii=False)

df_tie.loc[:, "prompt"] = df_tie["prompt"].apply(ensure_json_loads)
df_tie.loc[:, "response_a"] = df_tie["response_a"].apply(ensure_json_loads)
df_tie.loc[:, "response_b"] = df_tie["response_b"].apply(ensure_json_loads)
df_tie.head()

,prompt,model_a,model_b,difference,winner_model_a,winner_model_b,winner_tie,response_a,response_b
0,"[""Incorporating birth/death dates, accomplishm...",gpt-3.5-turbo,vicuna-33b,0.0,0,0,1,"[""You can create a program to generate a compr...","[""To create a program that generates a compreh..."
1,"[""Let's play a puzzle game! Can you connect th...",wizardlm-70b,llama-2-7b-chat,0.0,0,0,1,"[""Absolutely, I'd be happy to explain how thes...","[""Of course, I'd be happy to help you solve th..."
2,"[""Can you write a Ruby program to classify a g...",wizardlm-13b,ultralm-13b,0.0,0,0,1,"[""This Ruby code uses the TensorFlow framework...","[""Hello! Yes, I can help you write a Ruby prog..."
3,"[""Using Python, create a program that analyzes...",llama-2-70b-chat,llama-2-13b-chat,0.0,0,0,1,"[""Welcome to your AI assistant! I'm here to he...","[""Certainly! I'd be happy to help you with you..."
4,"[""Imagine a situation where you are solving a ...",llama-2-13b-chat,gpt-4,0.0,0,0,1,"[""Sure, I'd be happy to help you solve the cro...","[""A suitable word to fill in the blank for thi..."


In [14]:
df_tie_drop_duplicated_prompts = df_tie.loc[df_tie.groupby("prompt")["difference"].idxmax()]
print(df_tie_drop_duplicated_prompts.shape)
df_tie_drop_duplicated_prompts.reset_index(drop=True, inplace=True)
df_tie_drop_duplicated_prompts.head()

(16142, 9)


,prompt,model_a,model_b,difference,winner_model_a,winner_model_b,winner_tie,response_a,response_b
0,"["" Given a sentence in the English language, t...",starchat,ultralm-65b,0.0,0,0,1,"[""In the Nepali language, the sentence would b...","[""<translation>मोटे मुलुखत प्रतियोगितय अर्थशास..."
1,"["" You are given a background paragraph that d...",llama-2-70b-chat,vicuna-33b,0.0,0,0,1,"[""Output: David is a personal trainer. He has ...","[""David is a marathon runner. One day, he star..."
2,"[""!Hi Chat, I want to develop a python script ...",gpt-3.5-turbo,gpt-4,0.0,0,0,1,"[""Sure! I can help you get started with a Pyth...","[""Here is a Python script that will help you u..."
3,"[""# Exercise: Let's make an Order Dashboard!\n...",bard,gpt-3.5-turbo,0.0,0,0,1,"[""Here are the steps I would take to build the...","[""Thank you for providing the detailed instruc..."
4,"[""###\nLatify helps users on their healthy eat...",llama-2-13b-chat,mpt-30b-chat,0.0,0,0,1,"[""Hello! I'm here to assist you in any way I c...","[""Meno Assist is an app designed to assist wom..."


In [15]:
print(df_tie_drop_duplicated_prompts["prompt"][0])
print(df_tie_drop_duplicated_prompts["prompt"][1])

[" Given a sentence in the English language, translate it into the Nepali language.\nQ: Text: In capitalist economic structures, supply and demand is an economic model of price determination in a market.\n\nA: "]
[" You are given a background paragraph that describes one or more causal or physical relationships, such as a relationship in economics, scientific law, or nature. You need to come up with a fictional story that makes use of the concepts or the relationship described in the provided paragraph. Make sure that your story does not repeat the relation in the given paragraph (e.g., by avoiding verbatim copies of the given paragraph). Start by reading the background paragraph, and determine what relationship(s) are in the paragraph. Write a story that involves the relationship(s) that you determined are in the background paragraph. Try to use your own words as much as possible and not repeat the background paragraph. Feel free to make up places and events as long as they tie back t

In [16]:
df_tie_drop_duplicated_prompts.to_csv("kaggle-ultrafeedback-ties-drop-duplicate.csv", index=False)
df_tie.to_csv("kaggle-ultrafeedback-ties.csv", index=False)

In [17]:
def unicode_preprocess(input_str):
    return json.loads(input_str)

df_tie.loc[:, "prompt"] = df_tie["prompt"].apply(unicode_preprocess)
df_tie.loc[:, "response_a"] = df_tie["response_a"].apply(unicode_preprocess)
df_tie.loc[:, "response_b"] = df_tie["response_b"].apply(unicode_preprocess)

In [18]:
type(df_tie["response_a"][0])

list